# **Important Note**
The wine data set used for the below algorithm needed a very minor change in the class label names. i.e The Spaces in the names have been removed. 
**Example :  "residual sugar" is renamed as "residualsugar"**

The csv file with modified labels is attached in the zip file
 

In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
path=r"C:\Users\Prasanna\Desktop\wine-dataset1.csv"
df=pd.read_csv(path)
df.head()

,fixedacidity,volatileacidity,citricacid,residualsugar,chlorides,freesulfurdioxide,totalsulfurdioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0


# Function Definitions

In [3]:
def Is_Pure(data): #To detect the leaf
    
    quality_column = data[:, -1] #extracting only the labels (i.e quality column in our data set)
    unique_qualities = np.unique(quality_column)
    if len(unique_qualities) == 1:
        return True
    else:
        return False

In [4]:
def ClassForPureData(data):  #This function is used to return the class label of the pure data set or it can also be used when we need to pick majority class from a data
    
    quality_column = data[:, -1] 
    classlabels,classcounts=np.unique(quality_column,return_counts=True)
    majorityclassindex= np.argmax(classcounts)
    classi= classlabels[majorityclassindex]

    return classi

In [5]:
def splits(data): #To find the feasible split values under a column label
    
    splits = {} #creating an empty dictionary to store the splits with key of the dictionary as attribute index and values as various splits
    row,col = data.shape #to get the number of columns to iterate over the columns
    for i in range(col- 1):        # excluding the last column which is the quality label
        splits[i] = []
        tot_col = data[:, i] #to extract the total column of ith attribute
        uni_val = np.unique(tot_col)  #to weed out the repeated or duplicate values of the column

        for index in range(len(uni_val)):
            if index != 0: #to exclude first element as it does not have a previous value
                cr_val = uni_val[index]
                prev_val = uni_val[index - 1]
                split_value = (cr_val + prev_val) / 2
                
                splits[i].append(split_value)
    
    return splits

In [6]:
def split_data(data, column_to_split, value_to_split): #This fuction splits the data into two halves based on a value to split 
    
    split_column_values = data[:, column_to_split]

    left_data = data[split_column_values <= value_to_split]
    right_data = data[split_column_values > value_to_split]
    
    return left_data, right_data

In [7]:
def entropy(data):
    
    quality_column = data[:, -1]
    classlabels, classcounts = np.unique(quality_column, return_counts=True) #extract the counts of classes in dataset
    
    #numpy does element wise operations. So probabilities and entropy can be calculated as
    probab = classcounts/ classcounts.sum()
    entropy = sum(probab * -np.log2(probab))
     
    return entropy

In [8]:
def weighted_entropy(ldata, rdata):
    
    tot_data = len(ldata) + len(rdata)
    prob_ldata = len(ldata) / tot_data
    prob_rdata = len(rdata) / tot_data

    weigh_entro =  (prob_ldata * entropy(ldata) 
                      + prob_rdata * entropy(rdata))
    
    return weigh_entro

In [9]:
def best_split_entropy(data, potential_splits):
    
    arbit_entropy = 100
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_left, data_right = split_data(data, column_to_split=column_index, value_to_split=value)
            presentweigh_entropy = weighted_entropy(data_left, data_right)

            if presentweigh_entropy <= arbit_entropy:
                arbit_entropy = presentweigh_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [10]:
def splitTrainandTest(df, testdatasize):

    testdatasize = round(testdatasize * len(df))

    indices = df.index.tolist()
    test_points = random.sample(population=indices, k=testdatasize)

    test_set = df.loc[test_points]
    train_set = df.drop(test_points)
    
    return train_set, test_set

In [11]:
def dtree_algo_entropy(df, flag=0):
    
    global column_labels #To extract the column names in the form of strings and replace in the model instead of column index
    if flag == 0: # First time the data enters as pandas dataframe and it needs to be converted to numpy 2D array for working with functions 
        column_labels = df.columns
        data = df.values
    else:
        data = df           
    
    if Is_Pure(data): #This acts as the base case if the data contains only one class
        classi = ClassForPureData(data) #identifies which class 
        return classi

    else:     #The recursive part starts here if the data is not pure.
        feasible_splits = splits(data)
        split_column, split_value = best_split_entropy(data, feasible_splits)
        l_data, r_data = split_data(data, split_column, split_value)
        flag=flag+1
        
        feature_name = column_labels[split_column]
        question = "{} <= {}".format(feature_name, split_value)
        sub_tree = {question: []}
    
        yes_answer = dtree_algo_entropy(l_data, flag)
        no_answer = dtree_algo_entropy(r_data, flag)
        
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [12]:
def classify_datapoint(data_point, model):  #This function gets input as each data point(i.e all the 11 features) and says the classification
    ques = list(model.keys())[0]  #The tree is basically a dictionary with contains of a key and value basically. Here we are picking the first key which will be the dictionary
    col_name, operator, value = ques.split()

    if data_point[col_name] <= float(value):
        ans = model[ques][0]                   #note: general form of tree is tree={question,[positive_answer,negative_answer]}
    else:
        ans = model[ques][1]

    if isinstance(ans, dict): #checking if we narrow down at a class label or again a question in the form of dictionary
        residual_tree = ans
        return classify_datapoint(data_point, residual_tree)
    else:
      return ans
        

In [13]:
def calculate_accuracy(df, model):

    df["predicted_class"] = df.apply(classify_datapoint, axis=1, args=(model,)) #Adds a column named "predicted class" in the data frame which consists of predicted class of all test data
    df["actual_predicted_compare"] = df.predicted_class == df.quality #compares the predicted class column with the actual quality column
    
    accuracy = df.actual_predicted_compare.mean()
    
    return accuracy

In [14]:
random.seed(0) #random seed is set to have same training and testing set for implementing tree with gini index too
train_d,test_d= splitTrainandTest(df, testdatasize=0.4)
train_d.head()

,fixedacidity,volatileacidity,citricacid,residualsugar,chlorides,freesulfurdioxide,totalsulfurdioxide,density,pH,sulphates,alcohol,quality
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0
5,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,0
6,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,0
7,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,0
8,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,0


In [15]:
dtree_entropy= dtree_algo_entropy(train_d)
dtree_entropy

{'alcohol <= 10.850000000000001': [{'volatileacidity <= 0.2025': [{'density <= 0.99788': [{'totalsulfurdioxide <= 140.5': [{'freesulfurdioxide <= 27.5': [{'pH <= 3.565': [{'fixedacidity <= 6.55': [{'chlorides <= 0.0265': [1.0,
                0.0]},
              {'pH <= 3.145': [{'sulphates <= 0.375': [{'totalsulfurdioxide <= 98.0': [0.0,
                    {'pH <= 2.91': [0.0, 1.0]}]},
                  0.0]},
                {'pH <= 3.165': [1.0,
                  {'chlorides <= 0.0515': [{'sulphates <= 0.505': [{'residualsugar <= 1.7000000000000002': [0.0,
                        1.0]},
                      {'density <= 0.993865': [{'freesulfurdioxide <= 19.5': [0.0,
                          1.0]},
                        0.0]}]},
                    0.0]}]}]}]},
            1.0]},
          {'sulphates <= 0.53': [{'volatileacidity <= 0.155': [{'alcohol <= 10.05': [{'fixedacidity <= 6.75': [0.0,
                  {'sulphates <= 0.44': [{'residualsugar <= 10.100000000000001': [0.

In [16]:
calculate_accuracy(test_d,dtree_entropy)

0.8203164880040837

# **Qn.3b) K-Fold Cross Validation**

In [17]:
def cross_validation_split(dataset, folds):
        dataset_split = []
        df_copy = dataset
        fold_size = int(df_copy.shape[0] / folds)
        
        # for loop to save each fold
        for i in range(folds):
            fold = []
            # while loop to add elements to the folds
            while len(fold) < fold_size:
                # select a random element
                r = random.randrange(df_copy.shape[0])
                # determine the index of this element 
                index = df_copy.index[r]
                # save the randomly selected line 
                fold.append(df_copy.loc[index].values.tolist())
                # delete the randomly selected line from
                # dataframe not to select again
                df_copy = df_copy.drop(index)
            # save the fold     
            dataset_split.append(np.asarray(fold))
            
        return dataset_split 

In [18]:
#the cross_validation_split function returns a list which contains numpy 2d arrays of each fold
#df_algos receive the data set as dataframe(pandas). Inside the function, we are converting it to 2d array
cvl=cross_validation_split(df,10) #To get the folds. The cross validation split function returns 
cvl[9].shape

(489, 12)

The below section consists of creating training and testing data from the folds obtained. i.e For the first time, chunks 1-9 is treated as train data and 10th chunk as test data. For the second time, chunks 1,2,3,4,5,6,7,8,10 are treated as train data and chunk 9 is treated as test data. Similarly ten such training dataframe and testing data frame is created

In [19]:
#the folds are joined using concat function to create the train data. They are converted to pandas from numpy 2D as the input argument passed to decision tree algorithm function is a pandas data frame
concat1= np.concatenate((cvl[0],cvl[1],cvl[2],cvl[3],cvl[4],cvl[5],cvl[6],cvl[7],cvl[8]),axis=0)
tr_dframe1 = pd.DataFrame(concat1, columns = ['fixedacidity','volatileacidity','citricacid','residualsugar','chlorides','freesulfurdioxide','totalsulfurdioxide','density','pH','sulphates','alcohol','quality'])
te_dframe1 = pd.DataFrame(cvl[9], columns = ['fixedacidity','volatileacidity','citricacid','residualsugar','chlorides','freesulfurdioxide','totalsulfurdioxide','density','pH','sulphates','alcohol','quality'])

In [20]:
concat2= np.concatenate((cvl[0],cvl[1],cvl[2],cvl[3],cvl[4],cvl[5],cvl[6],cvl[7],cvl[9]),axis=0)
tr_dframe2 = pd.DataFrame(concat2, columns = ['fixedacidity','volatileacidity','citricacid','residualsugar','chlorides','freesulfurdioxide','totalsulfurdioxide','density','pH','sulphates','alcohol','quality'])
te_dframe2 = pd.DataFrame(cvl[8], columns = ['fixedacidity','volatileacidity','citricacid','residualsugar','chlorides','freesulfurdioxide','totalsulfurdioxide','density','pH','sulphates','alcohol','quality'])

In [21]:
concat3= np.concatenate((cvl[0],cvl[1],cvl[2],cvl[3],cvl[4],cvl[5],cvl[6],cvl[8],cvl[9]),axis=0)
tr_dframe3 = pd.DataFrame(concat3, columns = ['fixedacidity','volatileacidity','citricacid','residualsugar','chlorides','freesulfurdioxide','totalsulfurdioxide','density','pH','sulphates','alcohol','quality'])
te_dframe3 = pd.DataFrame(cvl[7], columns = ['fixedacidity','volatileacidity','citricacid','residualsugar','chlorides','freesulfurdioxide','totalsulfurdioxide','density','pH','sulphates','alcohol','quality'])

In [22]:
concat4= np.concatenate((cvl[0],cvl[1],cvl[2],cvl[3],cvl[4],cvl[5],cvl[7],cvl[8],cvl[9]),axis=0)
tr_dframe4 = pd.DataFrame(concat4, columns = ['fixedacidity','volatileacidity','citricacid','residualsugar','chlorides','freesulfurdioxide','totalsulfurdioxide','density','pH','sulphates','alcohol','quality'])
te_dframe4 = pd.DataFrame(cvl[6], columns = ['fixedacidity','volatileacidity','citricacid','residualsugar','chlorides','freesulfurdioxide','totalsulfurdioxide','density','pH','sulphates','alcohol','quality'])

In [23]:
concat5= np.concatenate((cvl[0],cvl[1],cvl[2],cvl[3],cvl[4],cvl[6],cvl[7],cvl[8],cvl[9]),axis=0)
tr_dframe5 = pd.DataFrame(concat5, columns = ['fixedacidity','volatileacidity','citricacid','residualsugar','chlorides','freesulfurdioxide','totalsulfurdioxide','density','pH','sulphates','alcohol','quality'])
te_dframe5 = pd.DataFrame(cvl[5], columns = ['fixedacidity','volatileacidity','citricacid','residualsugar','chlorides','freesulfurdioxide','totalsulfurdioxide','density','pH','sulphates','alcohol','quality'])

In [24]:
concat6= np.concatenate((cvl[0],cvl[1],cvl[2],cvl[3],cvl[5],cvl[6],cvl[7],cvl[8],cvl[9]),axis=0)
tr_dframe6 = pd.DataFrame(concat6, columns = ['fixedacidity','volatileacidity','citricacid','residualsugar','chlorides','freesulfurdioxide','totalsulfurdioxide','density','pH','sulphates','alcohol','quality'])
te_dframe6 = pd.DataFrame(cvl[4], columns = ['fixedacidity','volatileacidity','citricacid','residualsugar','chlorides','freesulfurdioxide','totalsulfurdioxide','density','pH','sulphates','alcohol','quality'])

In [25]:
concat7= np.concatenate((cvl[0],cvl[1],cvl[2],cvl[4],cvl[5],cvl[6],cvl[7],cvl[8],cvl[9]),axis=0)
tr_dframe7 = pd.DataFrame(concat7, columns = ['fixedacidity','volatileacidity','citricacid','residualsugar','chlorides','freesulfurdioxide','totalsulfurdioxide','density','pH','sulphates','alcohol','quality'])
te_dframe7 = pd.DataFrame(cvl[3], columns = ['fixedacidity','volatileacidity','citricacid','residualsugar','chlorides','freesulfurdioxide','totalsulfurdioxide','density','pH','sulphates','alcohol','quality'])

In [26]:
concat8= np.concatenate((cvl[0],cvl[1],cvl[3],cvl[4],cvl[5],cvl[6],cvl[7],cvl[8],cvl[9]),axis=0)
tr_dframe8 = pd.DataFrame(concat8, columns = ['fixedacidity','volatileacidity','citricacid','residualsugar','chlorides','freesulfurdioxide','totalsulfurdioxide','density','pH','sulphates','alcohol','quality'])
te_dframe8 = pd.DataFrame(cvl[2], columns = ['fixedacidity','volatileacidity','citricacid','residualsugar','chlorides','freesulfurdioxide','totalsulfurdioxide','density','pH','sulphates','alcohol','quality'])

In [27]:
concat9= np.concatenate((cvl[0],cvl[2],cvl[3],cvl[4],cvl[5],cvl[6],cvl[7],cvl[8],cvl[9]),axis=0)
tr_dframe9 = pd.DataFrame(concat9, columns = ['fixedacidity','volatileacidity','citricacid','residualsugar','chlorides','freesulfurdioxide','totalsulfurdioxide','density','pH','sulphates','alcohol','quality'])
te_dframe9 = pd.DataFrame(cvl[1], columns = ['fixedacidity','volatileacidity','citricacid','residualsugar','chlorides','freesulfurdioxide','totalsulfurdioxide','density','pH','sulphates','alcohol','quality'])

In [28]:
concat10= np.concatenate((cvl[1],cvl[2],cvl[3],cvl[4],cvl[5],cvl[6],cvl[7],cvl[8],cvl[9]),axis=0)
tr_dframe10 = pd.DataFrame(concat10, columns = ['fixedacidity','volatileacidity','citricacid','residualsugar','chlorides','freesulfurdioxide','totalsulfurdioxide','density','pH','sulphates','alcohol','quality'])
te_dframe10 = pd.DataFrame(cvl[0], columns = ['fixedacidity','volatileacidity','citricacid','residualsugar','chlorides','freesulfurdioxide','totalsulfurdioxide','density','pH','sulphates','alcohol','quality'])

**Training the folds below: [This may take 2 minutes]**

In [29]:
dtree_kf1= dtree_algo_entropy(tr_dframe1)
dtree_kf2= dtree_algo_entropy(tr_dframe2)
dtree_kf3= dtree_algo_entropy(tr_dframe3)
dtree_kf4= dtree_algo_entropy(tr_dframe4)
dtree_kf5= dtree_algo_entropy(tr_dframe5)
dtree_kf6= dtree_algo_entropy(tr_dframe6)
dtree_kf7= dtree_algo_entropy(tr_dframe7)
dtree_kf8= dtree_algo_entropy(tr_dframe8)
dtree_kf9= dtree_algo_entropy(tr_dframe9)
dtree_kf10= dtree_algo_entropy(tr_dframe10)

**Calculating the Accuracy for each tree built**

In [30]:
a1=calculate_accuracy(te_dframe1,dtree_kf1)
a2=calculate_accuracy(te_dframe2,dtree_kf2)
a3=calculate_accuracy(te_dframe3,dtree_kf3)
a4=calculate_accuracy(te_dframe4,dtree_kf4)
a5=calculate_accuracy(te_dframe5,dtree_kf5)
a6=calculate_accuracy(te_dframe6,dtree_kf6)
a7=calculate_accuracy(te_dframe7,dtree_kf7)
a8=calculate_accuracy(te_dframe8,dtree_kf8)
a9=calculate_accuracy(te_dframe9,dtree_kf9)
a10=calculate_accuracy(te_dframe10,dtree_kf10)

accu_list=[a1,a2,a3,a4,a5,a6,a7,a8,a9,a10]
accu_list

max_accuracy= max(accu_list)
min_accuracy= min(accu_list)

mean_accuracy= sum(accu_list)/10

print(max_accuracy,min_accuracy,mean_accuracy)

0.8813905930470347 0.820040899795501 0.8388548057259714


## **Improvement Strategies**

# Gini Index

In [31]:
def gini(data):
    
    quality_column = data[:, -1]
    classlabels, classcounts = np.unique(quality_column, return_counts=True) #extract the counts of classes in dataset
    
    #numpy does element wise operations. So probabilities and entropy can be calculated as
    probab = classcounts/ classcounts.sum()
    gini_score = sum(probab * (1-probab))
     
    return gini_score

In [32]:
def weighted_gini(ldata, rdata):
    
    tot_data = len(ldata) + len(rdata)
    prob_ldata = len(ldata) / tot_data
    prob_rdata = len(rdata) / tot_data

    weigh_gini =  (prob_ldata * gini(ldata) 
                      + prob_rdata * gini(rdata))
    
    return weigh_gini

In [33]:
def best_split_gini(data, potential_splits):
    
    arbit_gini = 100
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_left, data_right = split_data(data, column_to_split=column_index, value_to_split=value)
            presentweigh_gini = weighted_gini(data_left, data_right)

            if presentweigh_gini <= arbit_gini:
                arbit_gini = presentweigh_gini
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [34]:
def dtree_algo_gini(df, flag=0):
    
    global column_labels #To extract the column names in the form of strings and replace in the model instead of column index
    if flag == 0: # First time the data enters as pandas dataframe and it needs to be converted to numpy 2D array for working with functions 
        column_labels = df.columns
        data = df.values
    else:
        data = df           
    
    if Is_Pure(data): #This acts as the base case if the data contains only one class
        classi = ClassForPureData(data) #identifies which class 
        return classi

    else:     #The recursive part starts here if the data is not pure.
        feasible_splits = splits(data)
        split_column, split_value = best_split_gini(data, feasible_splits)
        l_data, r_data = split_data(data, split_column, split_value)
        flag=flag+1
        
        feature_name = column_labels[split_column]
        question = "{} <= {}".format(feature_name, split_value)
        sub_tree = {question: []}
    
        yes_answer = dtree_algo_gini(l_data, flag)
        no_answer = dtree_algo_gini(r_data, flag)
        
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [35]:
random.seed(0)
train_d,test_d= splitTrainandTest(df, testdatasize=0.4)
train_d.head()

,fixedacidity,volatileacidity,citricacid,residualsugar,chlorides,freesulfurdioxide,totalsulfurdioxide,density,pH,sulphates,alcohol,quality
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0
5,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,0
6,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,0
7,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,0
8,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,0


In [36]:
dtree_gini= dtree_algo_gini(train_d)
dtree_gini

{'alcohol <= 10.850000000000001': [{'volatileacidity <= 0.2025': [{'density <= 0.99788': [{'alcohol <= 10.116666665': [{'sulphates <= 0.875': [{'fixedacidity <= 6.55': [{'chlorides <= 0.0905': [0.0,
              {'sulphates <= 0.48': [0.0, 1.0]}]},
            {'density <= 0.99365': [{'pH <= 3.145': [{'density <= 0.992445': [{'alcohol <= 10.05': [0.0,
                    1.0]},
                  0.0]},
                {'alcohol <= 9.25': [0.0,
                  {'pH <= 3.3449999999999998': [1.0, 0.0]}]}]},
              {'residualsugar <= 10.25': [{'chlorides <= 0.0455': [{'alcohol <= 9.1': [1.0,
                    {'sulphates <= 0.44': [0.0,
                      {'freesulfurdioxide <= 27.5': [0.0,
                        {'density <= 0.99535': [1.0, 0.0]}]}]}]},
                  0.0]},
                {'fixedacidity <= 6.9': [0.0,
                  {'density <= 0.997385': [{'density <= 0.9967': [{'alcohol <= 9.75': [0.0,
                        1.0]},
                      1.0]},


In [37]:
calculate_accuracy(test_d,dtree_gini)

0.8065339458907606

The accuracy of the tree using gini index is coming to lesser than tree using entropy index.